# Data Transformation

In this section we will look at how to get our data into an appropriate format that we require. We will look at filtering, reshaping and aggregating data.

This section will cover key functions:
- Filtering data:  `iloc`, `query`
- Adding and modifying columns: `rename`, `drop`, `split`
- Reshaping data:  `pivot_table`, `melt`, `merge`, `concatenate`
- Aggregating data: `groupby`
- Row-wise operations: `apply`, `iterrows`

## Filtering Data

First we want to show how we can select certain columns. It is useful to know that if we just want to print the columns of a dataframe we can use built in `columns`

In [ ]:
import pandas as pd
from pathlib import Path

data = pd.read_excel(Path.cwd() / "data/Financial Sample.xlsx", header =2)

print(data.columns)


We can select certain columns based on their name or their positional argument.

In [ ]:
data["Segment"]

In [ ]:
# Must use double square brackets when referencing more than 1 column
data[["Segment","Country"]]

In [ ]:
# Referencing positional arguments this prints column 4 and 5
data.iloc[:,3:5]

Similarly you can use `iloc` for filtering rows where we have `.iloc[row_indexer, column_indexer]` 

In [ ]:
# This gives the second row of data
data.iloc[1,:]

In [ ]:
# This gives the last row of data
data.iloc[-1,:]

You can also filter rows based on certain conditions 
- Comparing values (`==`, `!=`, `>`, `<`, `>=`, `<=`)
- Comparing variables (`is`, `is not`)

In [ ]:
data[data["Discount Band"] == "High"]


In [ ]:
# Multiple conditions, each condition must be in brackets ()  - & is intersection operator, both conditions have to apply

data[(data["Discounts"] >= 10) & (data["Discounts"] <= 100)]

In [ ]:
# | is union operator - either condition can apply

data[(data["Country"] == "France") | (data["Discounts"] <= 100)]

Similarly we can use the `query` function to get the same result and can be easier to write.

In [ ]:
data.query('Country == "France" or Discounts <= 100')

You can also filter based on a list of possible entries using the `isin` function.

In [ ]:
special_segments = ["Government","Enterprise"]

data[data["Segment"].isin(special_segments)]

## Adding and modifying columns

You can simply add columns by stating it like you would access an exising column and equate it whatever you want this column to contain.

In [ ]:
data["Sale Price in 2025"] = data["Sale Price"] * 1.1

print(data[["Sale Price","Sale Price in 2025"]])

You can rename columns using the `rename` function and setting old column name and new column name out like a dictionary.

In [ ]:
data = data.rename(columns={"Segment":"Category","Sale Price in 2025":"SALE PRICE"})

print(data.columns)

You can remove whole columns using the `drop` function.

In [ ]:
data = data.drop(columns={"SALE PRICE"})

print(data.columns)

Both `rename` and `drop` include an argument `inplace` where `inplace=True` will alter the DataFrame in place, however it is better practice to reassign this DataFrame.

You might also wish to split up a column for example you may wish to split the "Date" column into its separate componants.

In [ ]:
data[["Year","Month","Day"]] = data["Date"].astype(str).str.split("-", expand=True)

print(data[["Year","Month","Day","Date"]])

You may only want part of the split, you can do this by treating the split like a list of the different components, so to just get the year this is the first item in the list so index 0.

In [ ]:
data["Year"] = data["Date"].astype(str).str.split("-", expand=True)[0]

print(data[["Year","Date"]])

## Reshaping data

### Pivot tables

A pivot table is a really useful way to reshape and summarise data. You choose a column to be the index, this means each row will represent a unique entry from this column, then you pick another column to the columns, this means each columns will represent a unique entry from this column, then you choose the values, these will be entries of the table and this must be a numeric column. We can use pandas `pivot_table` function.

In [ ]:
df_pivot = data.pivot_table(index="Date", columns="Country", values="Units Sold", aggfunc="sum").reset_index()

print(df_pivot)

### Melting (Unpivoting)

Now suppose we the data so we have only one value on each row and every row is a unique combination of Date and Country. We can unpivot the table using the `melt` function.

In [ ]:
df_unpivot = df_pivot.melt(id_vars=["Date"])

print(df_unpivot)

### Merging DataFrames

You can combine 2 DataFrames together based on 1 or more common keys using built in pandas function `merge`. With argument how you have the option of right, left, inner and outer.

In [ ]:
df1 = pd.read_excel(Path.cwd() / "data/Financial Sample.xlsx", header =2)
df2 = pd.read_csv(Path.cwd() / "data/customers-1000.csv")

df_merged = pd.merge(df1, df2, on='Country', how='inner')

print(df_merged)

### Concatenating DataFrames

If you want to combine 2 dataframes together that have the same columns by stacking one on top of the other, this is what the `concatenate` function is for

In [ ]:
df3 = pd.read_csv(Path.cwd() / "data/customers-1000.csv", usecols=[2,3])
df3["variable"] = "A"
df4 = pd.read_csv(Path.cwd() / "data/customers-1000.csv", usecols=[2,3])
df4["variable"] = "B"

df_concat = pd.concat([df3,df4], ignore_index=True)

print(df_concat)

## Aggregating Data

Say you want to combine rows together and apply some sort of calculation to the rows you are combining such as sum or mean. You can group rows together using `groupby`.

In [ ]:
data = pd.read_excel(Path.cwd() / "data/Financial Sample.xlsx", header =2, usecols=[1,4,7])

df_grouped = data.groupby(['Country'])["Units Sold"].sum()

print(df_grouped)

In [ ]:
data = pd.read_excel(Path.cwd() / "data/Financial Sample.xlsx", header =2, usecols=[1,4,7])

df_grouped = data.groupby(['Country'])["Gross Sales"].mean()

print(df_grouped)

## Row-Wise Operations

We have considered vectorized operation where we can apply a function to the entire column of a dataframe at once. This is much more efficient and reduces computational costs but there are situations where you will need to apply functions a row at a time. This is where the `apply` function comes in very useful.

In [58]:
# Define a custom function to return the gross sales per unit
def get_sales_per_unit(row):
    return row['Gross Sales'] / row['Units Sold']

In [ ]:
data['Sales per unit'] = data.apply(get_sales_per_unit, axis=1)

data

Suppose you want to keep this more concise and you won't be using this function anywhere else, it is better to use a lambda function.

In [ ]:
data['Sales per unit concise'] = data.apply(lambda row: row['Gross Sales'] / row['Units Sold'], axis=1)

data

There will also be scenarios were we want to iterate through rows of a dataframe using `iterrows` function. This allows you to iterate through index and row pairs.

In [ ]:
performance_categories = []

# Iterate over the DataFrame rows
for index, row in data.iterrows():
    if row['Units Sold'] > 1000:
        performance_categories.append('High')
    elif 700 <= row['Units Sold'] <= 1000:
        performance_categories.append('Medium')
    else:
        performance_categories.append('Low')

# Add the new column to the DataFrame
data['performance_category'] = performance_categories

data